In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import pie,axis,show
import seaborn as sns
import random as rd
import datetime as dt
from termcolor import colored
from datetime import date
from datetime import datetime

In [6]:
t1=pd.read_csv('../data/transactions_1.csv')
t2=pd.read_csv('../data/transactions_2.csv')
t3=pd.read_csv('../data/transactions_3.csv')
# concatenate the 3 datasets into 1 with all transactions
trans=pd.concat([t1, t2, t3]) 

print('Shape of the transactions dataset',trans.shape)
print('Number of null in each column',trans.isna().sum())
print(trans.sample(5))
trans.info()

Shape of the transactions dataset (2181026, 12)
Number of null in each column transaction_id                0
transactions_type             0
transactions_currency         0
amount_usd                    0
transactions_state            0
ea_cardholderpresence    933206
ea_merchant_mcc          923975
ea_merchant_city         924080
ea_merchant_country      924239
direction                     0
user_id                       0
created_date                  0
dtype: int64
             transaction_id transactions_type transactions_currency  \
238912  transaction_1738912      CARD_PAYMENT                   GBP   
63177   transaction_1563177          TRANSFER                   RON   
263172  transaction_1063172      CARD_PAYMENT                   EUR   
672157  transaction_2172157      CARD_PAYMENT                   EUR   
694036  transaction_1494036      CARD_PAYMENT                   GBP   

        amount_usd transactions_state ea_cardholderpresence  ea_merchant_mcc  \
238912        9.20

In [7]:
trans['created_date'] = pd.to_datetime(trans['created_date'])

In [8]:
trans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2181026 entries, 0 to 681025
Data columns (total 12 columns):
transaction_id           object
transactions_type        object
transactions_currency    object
amount_usd               float64
transactions_state       object
ea_cardholderpresence    object
ea_merchant_mcc          float64
ea_merchant_city         object
ea_merchant_country      object
direction                object
user_id                  object
created_date             datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(9)
memory usage: 216.3+ MB


In [9]:
trans=trans[trans.transactions_state=='COMPLETED']
print(trans['transactions_state'].value_counts())

COMPLETED    1916387
Name: transactions_state, dtype: int64


In [10]:
trans_grouped = trans.groupby(['user_id'], as_index=True)['amount_usd'].agg(['sum','count','mean'])
trans_grouped.rename(columns={'sum':'total_amount', 'count':'number_transactions','mean':'avg_amount'}, inplace=True)
trans_grouped.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14821 entries, user_0 to user_9999
Data columns (total 3 columns):
total_amount           14821 non-null float64
number_transactions    14821 non-null int64
avg_amount             14821 non-null float64
dtypes: float64(2), int64(1)
memory usage: 463.2+ KB


In [11]:
#trans_grouped.to_csv('trans_amount_vol.csv')

In [12]:
trans['created_date'].min()

Timestamp('2018-01-01 08:53:48.538709')

In [13]:
trans_grouped2 = trans.groupby(['user_id'], as_index=True)['created_date'].agg(['min','max'])
trans_grouped2.rename(columns={'min':'first_transaction', 'max':'last_transaction'}, inplace=True)
trans_grouped2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14821 entries, user_0 to user_9999
Data columns (total 2 columns):
first_transaction    14821 non-null datetime64[ns]
last_transaction     14821 non-null datetime64[ns]
dtypes: datetime64[ns](2)
memory usage: 347.4+ KB


In [14]:
trans_grouped2['transaction_period']=(trans_grouped2['last_transaction']-trans_grouped2['first_transaction'])
print(trans_grouped2.head(4))
trans_grouped2['transaction_period']=trans_grouped2['transaction_period'].dt.days +0.5
#['transaction_period'].days
print(trans_grouped2.head(4))

                  first_transaction           last_transaction  \
user_id                                                          
user_0   2018-09-10 18:54:08.965855 2019-05-10 05:09:13.868855   
user_1   2018-02-04 08:28:17.978146 2019-05-04 02:30:11.492146   
user_10  2018-06-29 15:38:15.855413 2019-05-11 13:07:42.624413   
user_100 2018-12-02 14:23:12.926151 2019-02-24 13:36:50.054151   

               transaction_period  
user_id                            
user_0   241 days 10:15:04.903000  
user_1   453 days 18:01:53.514000  
user_10  315 days 21:29:26.769000  
user_100  83 days 23:13:37.128000  
                  first_transaction           last_transaction  \
user_id                                                          
user_0   2018-09-10 18:54:08.965855 2019-05-10 05:09:13.868855   
user_1   2018-02-04 08:28:17.978146 2019-05-04 02:30:11.492146   
user_10  2018-06-29 15:38:15.855413 2019-05-11 13:07:42.624413   
user_100 2018-12-02 14:23:12.926151 2019-02-24 13:36:50.0

In [15]:
trans_grouped_all = pd.merge(trans_grouped, trans_grouped2, on='user_id', how='left')
trans_grouped_all.sample(5)

,total_amount,number_transactions,avg_amount,first_transaction,last_transaction,transaction_period
user_id,,,,,,
user_5026,5878.89,91,64.603187,2018-08-07 04:39:17.782632,2019-04-30 04:46:46.802632,266.5
user_14685,47.65,2,23.825000,2019-04-13 05:53:18.062003,2019-04-13 05:56:12.706003,0.5
user_3586,47458.54,18,2636.585556,2018-04-25 22:18:26.330770,2018-08-26 08:46:46.023770,122.5
user_1625,3320.87,52,63.862885,2018-08-10 02:27:33.446756,2019-05-06 11:13:16.285756,269.5
user_7175,15743.23,68,231.518088,2018-08-20 07:24:52.372033,2019-03-30 13:15:29.750033,222.5


In [16]:
#def period(var1):
 # return (var1[1]/var1[0])
#trans_grouped_all['periodicity_trans']=trans_grouped_all[['number_transactions','transaction_period']].apply(period)
#trans_grouped_all['periodicity_trans']=trans_grouped_all['transaction_period']/trans_grouped_all['number_transactions']
#trans_grouped_all['transactions_perday']=1/trans_grouped_all['periodicity_trans']

In [17]:
trans_grouped_all.head(4)

,total_amount,number_transactions,avg_amount,first_transaction,last_transaction,transaction_period
user_id,,,,,,
user_0,6925.08,520,13.317462,2018-09-10 18:54:08.965855,2019-05-10 05:09:13.868855,241.5
user_1,12955.54,133,97.410075,2018-02-04 08:28:17.978146,2019-05-04 02:30:11.492146,453.5
user_10,5438.92,123,44.218862,2018-06-29 15:38:15.855413,2019-05-11 13:07:42.624413,315.5
user_100,4230.10,55,76.910909,2018-12-02 14:23:12.926151,2019-02-24 13:36:50.054151,83.5


In [18]:
print(trans_grouped_all[trans_grouped_all['transaction_period']==0])

Empty DataFrame
Columns: [total_amount, number_transactions, avg_amount, first_transaction, last_transaction, transaction_period]
Index: []


In [19]:
print(trans_grouped_all.loc['user_10031',:])

total_amount                                 9.79
number_transactions                             2
avg_amount                                  4.895
first_transaction      2019-04-08 18:25:04.154064
last_transaction       2019-04-08 18:29:17.730064
transaction_period                            0.5
Name: user_10031, dtype: object


In [20]:
print(trans['created_date'].max())

2019-05-15 21:18:22.660896


In [21]:
# trans_grouped_all.to_csv('trans_amount_periods.csv')

In [23]:
us=pd.read_csv('../processed_data/users_devices.csv')

In [24]:
data = pd.merge(trans_grouped_all, us, on='user_id', how='left')
data.sample(5)

,user_id,total_amount,number_transactions,avg_amount,first_transaction,last_transaction,transaction_period,Australia,Central_Rich_Europe,East_Europe_Balkans,...,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_push,attributes_notifications_marketing_email,num_contacts,num_referrals,num_successful_referrals,brand,created_year,created_month
11152,user_6560,3688.38,389,9.481697,2018-07-27 23:51:32.216280,2019-05-10 18:26:53.284280,286.5,0,0,0,...,0,0,0.0,0.0,49,0,0,1,2018,7
3141,user_12957,54.93,5,10.986000,2018-07-22 12:08:18.351464,2019-03-05 11:44:09.093464,225.5,0,0,0,...,0,0,1.0,1.0,0,0,0,1,2018,7
11209,user_6613,104.10,23,4.526087,2018-11-27 23:16:36.807772,2019-04-27 20:25:05.893772,150.5,0,0,1,...,0,0,1.0,1.0,22,0,0,0,2018,11
9639,user_5116,432.55,26,16.636538,2018-03-30 09:00:31.585381,2019-04-19 18:17:33.760381,385.5,0,0,1,...,0,0,1.0,1.0,2,0,0,0,2018,3
351,user_10327,2772.70,60,46.211667,2018-11-26 23:39:50.133100,2019-05-11 01:50:41.140100,165.5,0,0,0,...,0,1,1.0,1.0,16,0,0,0,2018,11


In [25]:
# data.to_csv('trans_users_dev.csv')

In [27]:
pd.crosstab(trans['transactions_type'], trans['direction'])

direction,INBOUND,OUTBOUND
transactions_type,,
ATM,0,61518
CARD_PAYMENT,0,999418
CARD_REFUND,9208,0
CASHBACK,60278,0
EXCHANGE,0,124800
FEE,0,18786
REFUND,1193,0
TAX,0,2242
TOPUP,242621,0


In [50]:
trans = trans[trans.direction == 'OUTBOUND']
trans_direction = trans.groupby(['user_id'], as_index=True)['direction'].agg(['count'])
trans_direction.rename(columns={'count':'profit'}, inplace=True)
trans_direction.head(4)

# pd.crosstab(index = data['user_id'], columns = trans_direction['profit'])

,profit
user_id,
user_0,357
user_1,106
user_10,71
user_100,44
